In [1]:
import numpy as np
import pandas as pd

In [5]:
spam = pd.read_table('./data/SMSSpamCollection.txt', sep='\t', header=0)
spam.head()

,classe,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
type(spam)

pandas.core.frame.DataFrame

In [7]:
spam.shape

(5572, 2)

In [8]:
spam.describe()

,classe,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [11]:
pd.crosstab(index=spam['classe'], columns='count')

col_0,count
classe,
ham,4825
spam,747


In [12]:
from sklearn.model_selection import train_test_split
spamTrain, spamTest = train_test_split(spam, train_size=3572, random_state=1, stratify=spam['classe'])

In [14]:
freqTrain = pd.crosstab(index=spamTrain['classe'], columns='count')
freqTrain/freqTrain.sum()

col_0,count
classe,
ham,0.865901
spam,0.134099


In [15]:
freqTest = pd.crosstab(index=spamTest['classe'], columns='count')
freqTest/freqTest.sum()

col_0,count
classe,
ham,0.866
spam,0.134


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, stop_words='english')

XTrain = vectorizer.fit_transform(spamTrain['message'])

In [34]:
vectorizer.get_feature_names_out()

array(['00', '000', '000pes', ..., 'zouk', 'zyada', '〨ud'], dtype=object)

In [35]:
len(vectorizer.get_feature_names_out())

6789

In [36]:
mdtTrain = XTrain.toarray()

In [37]:
type(mdtTrain)

numpy.ndarray

In [38]:
mdtTrain.shape

(3572, 6789)

In [39]:
word_freq = np.sum(mdtTrain, axis=0)
word_freq

array([ 7, 16,  1, ...,  1,  1,  1], dtype=int64)

In [40]:
index = np.argsort(word_freq)
index

array([3394, 3820, 3817, ..., 5962, 6756, 6061], dtype=int64)

In [42]:
printstring = {'word': np.asarray(vectorizer.get_feature_names_out())[index], 'freq': word_freq[index]}
pd.DataFrame(printstring).tail(10)

,word,freq
6779,your,394
6780,for,399
6781,my,403
6782,me,454
6783,is,478
6784,and,522
6785,in,528
6786,the,647
6787,you,1042
6788,to,1091


In [43]:
# use logistic regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(XTrain, spamTrain['classe'])

LogisticRegression()

In [44]:
model.coef_.shape

(1, 6789)

In [45]:
model.intercept_

array([-4.9643061])

In [46]:
mdtTest = vectorizer.transform(spamTest['message'])
mdtTest.shape

(2000, 6789)

In [47]:
predTest = model.predict(mdtTest)

In [48]:
from sklearn import metrics

mcTest = metrics.confusion_matrix(spamTest['classe'], predTest)
mcTest

array([[1731,    1],
       [  40,  228]], dtype=int64)

In [49]:
metrics.recall_score(spamTest['classe'], predTest, pos_label='spam')


0.8507462686567164

In [50]:
# precision
metrics.precision_score(spamTest['classe'], predTest, pos_label='spam')

0.9956331877729258

In [51]:
# F1 score
metrics.f1_score(spamTest['classe'], predTest, pos_label='spam')

0.9175050301810864

In [52]:
# accuracy
metrics.accuracy_score(spamTest['classe'], predTest)

0.9795